# Dimensionality Reduction

## PCA
* PCA Discovers an axis rotation i.e. linear transformation.
* Uses the eigen decomposition of the (symmetric positive-semidefinite) covariance matrix.
* Prerequisite is mean centering.
* Finds an orthonormal basis. 
* In effect, recursively performs axis rotation to where remaining variance is maximized along next axis.
* Discovers linear transformations only. (But kernel trick can uncover non-linear ones.)
* Under the new coordinates, there is no covariance, and the covariance matrix == the diagonalized eigenvalue matrix.
* For dimensionality reduction (with lossy reconstruction), discard the axiis with the smallest eigenvalues.

The eigen decomposition of data matrix D:   
Scatter matrix = $D D^T$  
Covariance matrix = $\frac{D D^T}{n} - \mu^T \mu$ (mean centered)    
Covariance = $P \Lambda P^{-1}$   
This factorization is possible because cov is symmetric and positive semi-definite.

PCA vs SVD
* PCA is a special case of SVD. 
* Whenever all feature means are 0, SVD==PCA.
* PCA generates one basis for the matrix rows. SVD generates one for the rows and one for the columns. 


## SVD
* Does not require mean centering.
* Ideal for sparse non-negative matrices e.g. word vectors of documents.
* SVD can be transformed in spectral decomposition.

SVD discovers the latent factors and ranks them.
For example, if D = movie-patrons x movie-titles,
SVD discovers a smaller matrix of patron-preferences x movie-categories.

SVD provides a lossy reconstruction of the data.
Even better, the lost bits tend to be noise or outliers.

$D = Q \Sigma P^{-1}$  
SVD uses this factorization of the data matrix.   
This facorization always exists. 

D = data matrix on original axiis.   
Q = left singular matrix of orthonormal column vectors of $D^T$.  
$\Sigma$ = non-negative singular values along the diagonal (in decreasing order)  
P = right singular matrix of orthonormal column vectors of $D$.  

D = (n x d) = n instances x d dimensions.  
Q = (n x n)  
$\Sigma$ = (n x d) 
P = (d x d)  

$P^T P = I$  
$Q^T Q = I$  

Q = eigenvectors of $D D^T$  
P = eigenvectors of $D^T D$  

Number of non-zero entries in $\Sigma$
equals Rank(D) and is <= min(n,d).